In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import gdown
import zipfile
import os

In [43]:
# Loading main data from GDrive.
file_id = "1omkd2We20kQ5fmJBLvm46SARBBXEmWJI"
url = f"https://drive.google.com/uc?id={file_id}"
file_extension = "downloaded_file.csv"
gdown.download(url, file_extension, quiet=False)
df = pd.read_csv(file_extension, sep=",")

Downloading...
From (original): https://drive.google.com/uc?id=1omkd2We20kQ5fmJBLvm46SARBBXEmWJI
From (redirected): https://drive.google.com/uc?id=1omkd2We20kQ5fmJBLvm46SARBBXEmWJI&confirm=t&uuid=effe06e7-e314-4bc8-b9d5-98c539fd0fe7
To: C:\Users\KW\PycharmProjects\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 254M/254M [00:06<00:00, 38.7MB/s] 


### Main data preprocessing

In [44]:
merged_bdl = df.copy()

# More intuitive names.
columns_to_rename = {
    "REGON": "REGON_of_district",
    "wynagrodzenia": "bdl_avg_district_salary_per_year",
    "year_announced": "year",
    "promocja": "advertised_institution",
    "kandydaci": "applicants",
    "views": "ad_views",
    "JPT_NAZWA_": "district_name"
}

for key, value in columns_to_rename.items():
    merged_bdl.rename(columns={key: value}, inplace=True)

In [45]:
# Inconsistent data types (for later merge).
merged_bdl['district'] = merged_bdl['district'].astype(str)
merged_bdl['voivodeship'] = merged_bdl['voivodeship'].astype(str)
merged_bdl['year'] = merged_bdl['year'].astype(int)

### BDL data preprocessing


In [46]:
# Importing BDL data.

cena_mieszkan = pd.DataFrame()
mediana_wieku = pd.DataFrame()
saldo_migracji = pd.DataFrame() # dotyczy osób w wieku produkcyjnym
wsp_feminizacji = pd.DataFrame()
wydatki_powiat = pd.DataFrame() # w złotówkach
wynagrodzenia = pd.DataFrame() # w złotówkach
kapital_zagraniczny = pd.DataFrame()
bezrobocie = pd.DataFrame()

bdl_dfs_list = [
    cena_mieszkan,
    mediana_wieku,
    saldo_migracji,
    wsp_feminizacji,
    wydatki_powiat,
    wynagrodzenia,
    kapital_zagraniczny,
    bezrobocie
]

file_ids = {
    "cena_mieszkan": "12UWwTtqRazIEctdzxib34fs8I_-ZUOD7",
    "mediana_wieku": "1D8jvho56cqn5dNGhMPaHujF3tmoQ1L6K",
    "saldo_migracji": "1sVL3iXGzc9D2k38ogErl84mVIzl6SzIP",
    "wsp_feminizacji": "163IRO4q5iU-MdyN3kYXkFHDavpyr353f",
    "wydatki_powiat": "1YMXflzDmozNqxQLpcd6oc1e1vtUVyUtS",
    "wynagrodzenia": "104z3VjCcv66e2L034D-o3vJY2wROJwCY",
    "kapital_zagraniczny": "1cthFg7vwfyQdrHjce4MDCst3cktJaJq0",
    "bezrobocie": "1vi07rn-mKkDHaWVtQchGJhs0VQIKmFr5"
}

for i in range(len(file_ids)):
    url = f"https://drive.google.com/uc?id={list(file_ids.values())[i]}"
    file_extension = "downloaded_file.csv"
    gdown.download(url, file_extension, quiet=False)
    bdl_dfs_list[i] = pd.read_csv(file_extension, sep=";", on_bad_lines='skip')

Downloading...
From: https://drive.google.com/uc?id=12UWwTtqRazIEctdzxib34fs8I_-ZUOD7
To: C:\Users\KW\PycharmProjects\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 204k/204k [00:00<00:00, 4.23MB/s]
Downloading...
From: https://drive.google.com/uc?id=1D8jvho56cqn5dNGhMPaHujF3tmoQ1L6K
To: C:\Users\KW\PycharmProjects\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 144k/144k [00:00<00:00, 9.04MB/s]
Downloading...
From: https://drive.google.com/uc?id=1sVL3iXGzc9D2k38ogErl84mVIzl6SzIP
To: C:\Users\KW\PycharmProjects\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 259k/259k [00:00<00:00, 7.01MB/s]
Downloading...
From: https://drive.google.com/uc?id=163IRO4q5iU-MdyN3kYXkFHDavpyr353f
To: C:\Users\KW\PycharmProjects\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 176k/176k [00:00<00:00, 3.44MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YMXflzDmozNqxQLpcd6oc1e1vtUVyUtS
To: C:\Users\K

In [47]:
# Wybieram interesujące mnie kolumny dotyczące bezrobocia.
bdl_dfs_list[7] = pd.concat([bdl_dfs_list[7].iloc[:, [0]], bdl_dfs_list[7].iloc[:, 26:32]], axis=1)

# # Wybieram interesujące mnie kolumny dotyczące udziału kapitału zagranicznego.
bdl_dfs_list[6] = pd.concat([bdl_dfs_list[6].iloc[:, [0]], bdl_dfs_list[6].iloc[:, 14:20]], axis=1)

In [48]:
# Removing unnecessary columns.

unique_columns_to_delete = [
    ["Transakcje rynkowe", "Powierzchnia użytkowa lokali mieszkalnych"],
    ["Migracje", "Wiek"],
    ["Ogółem"],
    ["Wskaźniki"],
    ["Wyszczególnienie"],
    ["Płeć"]
]

for i in range(len(bdl_dfs_list)):
    try:
        bdl_dfs_list[i].drop(["Nazwa", "Jednostka miary", "Atrybut"], axis=1, inplace=True)
        bdl_dfs_list[i] = bdl_dfs_list[i].iloc[:, :-1] # the last column has a different name in every df and is always unnecessary.
    except:
        pass

    # Columns unique for single dataframes.
    for columns in unique_columns_to_delete:
        try:
            bdl_dfs_list[i].drop(columns, axis=1, inplace=True)
        except:
            pass

In [49]:
# Renaming columns with the same name across multiple dataframes.
try:
    for i in range(len(bdl_dfs_list)):
        bdl_dfs_list[i].rename(columns={"Wartosc": list(file_ids.keys())[i]}, inplace=True)
except:
    pass

In [50]:
# Cleaning non-absolute values of salaries.
bdl_dfs_list[5]["wynagrodzenia"] = (
    bdl_dfs_list[5]["wynagrodzenia"]
    .str.replace(",", ".", regex=False)
    .astype(float)
)

bdl_dfs_list[5] = bdl_dfs_list[5][bdl_dfs_list[5]["wynagrodzenia"] >= 2200]

In [51]:
bdl_dfs_list[6] = pd.melt(
    bdl_dfs_list[6],
    id_vars=['Kod'],
    value_vars=['kapitał zagraniczny na 1 mieszkańca w wieku produkcyjnym;2018;[zł]', 'kapitał zagraniczny na 1 mieszkańca w wieku produkcyjnym;2019;[zł]', 'kapitał zagraniczny na 1 mieszkańca w wieku produkcyjnym;2020;[zł]', 'kapitał zagraniczny na 1 mieszkańca w wieku produkcyjnym;2021;[zł]', 'kapitał zagraniczny na 1 mieszkańca w wieku produkcyjnym;2022;[zł]', 'kapitał zagraniczny na 1 mieszkańca w wieku produkcyjnym;2023;[zł]'],
    var_name='Rok',
    value_name='foreign_capital'
)

# Strip the 'value' prefix to keep only the year
bdl_dfs_list[6]['Rok'] = bdl_dfs_list[6]['Rok'].str.extract(r'(\d{4})')

In [52]:
bdl_dfs_list[7] = pd.melt(
    bdl_dfs_list[7],
    id_vars=['Kod'],
    value_vars=['w % ludności aktywnej zawodowo;2018;[%]', 'w % ludności aktywnej zawodowo;2019;[%]', 'w % ludności aktywnej zawodowo;2020;[%]', 'w % ludności aktywnej zawodowo;2021;[%]', 'w % ludności aktywnej zawodowo;2022;[%]', 'w % ludności aktywnej zawodowo;2023;[%]'],
    var_name='Rok',
    value_name='unemployment'
)

# Strip the 'value' prefix to keep only the year
bdl_dfs_list[7]['Rok'] = bdl_dfs_list[7]['Rok'].str.extract(r'(\d{4})')

### Merge

In [53]:
for i in range(len(bdl_dfs_list)):
    bdl_dfs_list[i]['Kod'] = bdl_dfs_list[i]['Kod'].astype(str)
    mask_whole_country_or_voivodeships = ((bdl_dfs_list[i]['Kod'].str[0] == "0") & (bdl_dfs_list[i]['Kod'].str[2] == "0") & (bdl_dfs_list[i]['Kod'].str[3] == "0"))
    bdl_dfs_list[i] = bdl_dfs_list[i][~mask_whole_country_or_voivodeships]

    # For single-digit voivodeships, the code's length is 6.
    mask_length_6 = (bdl_dfs_list[i]['Kod'].str.len() == 6) 
    mask_length_7 = (bdl_dfs_list[i]['Kod'].str.len() == 7)

    # Single-digits districts include 0 and we want to remove it (e.g. '07' instead of '7').
    mask_district_single_digit_length_6 = ((bdl_dfs_list[i]['Kod'].str[1] == "0") & mask_length_6)
    mask_district_non_single_digit_length_6 = ((bdl_dfs_list[i]['Kod'].str[1] != "0") & mask_length_6)
    mask_district_single_length_7 = ((bdl_dfs_list[i]['Kod'].str[2] == "0") & mask_length_7)
    mask_district_non_single_length_7 = ((bdl_dfs_list[i]['Kod'].str[2] != "0") & mask_length_7)
    
    # Districts.
    bdl_dfs_list[i].loc[mask_district_single_digit_length_6, 'district'] = bdl_dfs_list[i].loc[mask_district_single_digit_length_6, 'Kod'].str[2:3]
    bdl_dfs_list[i].loc[mask_district_non_single_digit_length_6, 'district'] = bdl_dfs_list[i].loc[mask_district_non_single_digit_length_6, 'Kod'].str[1:3]
    bdl_dfs_list[i].loc[mask_district_single_length_7, 'district'] = bdl_dfs_list[i].loc[mask_district_single_length_7, 'Kod'].str[3:4]
    bdl_dfs_list[i].loc[mask_district_non_single_length_7, 'district'] = bdl_dfs_list[i].loc[mask_district_non_single_length_7, 'Kod'].str[2:4]
    
    # Voivodeships.
    bdl_dfs_list[i].loc[mask_length_6, 'voivodeship'] = bdl_dfs_list[i].loc[mask_length_6, 'Kod'].str[0:1]
    bdl_dfs_list[i].loc[mask_length_7, 'voivodeship'] = bdl_dfs_list[i].loc[mask_length_7, 'Kod'].str[0:2]

    bdl_dfs_list[i]['Rok'] = bdl_dfs_list[i]['Rok'].astype(int)
    bdl_dfs_list[i].drop(['Kod'], axis=1, inplace=True)

    # Merge.
    merged_bdl = pd.merge(merged_bdl, bdl_dfs_list[i], left_on=["district", "voivodeship", "year"], right_on=["district", "voivodeship", "Rok"], how="left")
    merged_bdl.drop(["Rok"], axis=1, inplace=True)

### Post-merge data cleaning

In [54]:
# Removing 2017 data (too few records).
merged_bdl = merged_bdl[merged_bdl["year"] > 2017]

In [55]:
# There is no data about foreign capital for 3 districts; it seems that there is no foreign capital there.
merged_bdl.loc[merged_bdl['foreign_capital'].isnull(), 'foreign_capital'] = 0

In [56]:
# Oddly, GUS didn't publish recent data for Jastrzębie-Zdrój.
# no data found for 2022, 2023, imputing by calculating an average growth rate.
merged_bdl.loc[(merged_bdl['year'] == 2022) & (merged_bdl['city'] == 'Jastrzębie-Zdrój'), 'wynagrodzenia'] = 9259
merged_bdl.loc[(merged_bdl['year'] == 2023) & (merged_bdl['city'] == 'Jastrzębie-Zdrój'), 'wynagrodzenia'] = 9531